<h1> Setup </h1>


In [1]:
import matplotlib.pyplot as plt
import awkward as ak
import pandas as pd
import numpy as np
import uproot
from hist import Hist, axis
import os

#Constants
current_dir = os.getcwd()
tree_name= "Delphes"  #all of them are named delphes
luminescence = 300 #fb

#Signal

signal_folder = "Signals_DMSimpl_Spin0/"
signal_dir = os.path.join(current_dir, signal_folder)# Path to Signals_DMSimpl_Spin0 folder
signal_files = {     #times 1000 to convert from pb to fb
            "VBF_DM_MY0_100_MXD_10.root": 0.0038 * 1000,
            "VBF_DM_MY0_500_MXD_10.root" : 0.0086 * 1000,
            "VBF_DM_MY0_1000_MXD_10.root" :  0.0018* 1000,
            "VBF_DM_MY0_2000_MXD_10.root" : 0.00026 * 1000
            }


#Background
background_folders = {"BKG_Wjets_WToLNu" : 47744.85 * 1000,
                       "BKG_Zjets_ZToNuNu": 8818.65 * 1000}  #foldername, significance



#will store data to be used in final "cut chart"
cuts_strings = []
Signal_numevents = []
Background_numevents = []
significances = []
   


<h2> Bins </h2>

In [2]:
binning = {
    "PT": {"range": (20, 1000), "bins": 14},  # (300 - 20) / 20 = 14 bins
    "Eta*Eta": {"range": (-25, 25), "bins": 500},  # (25 - (-25)) / 0.1 = 500 bins
    "Delta_Eta": {"range": (0, 10), "bins": 100},  # (10 - 0) / 0.1 = 100 bins
    "Transverse": {"range": (0, 500), "bins": 100},  # (2000 - 500) / 20 = 75 bins
    "Eta": {"range": (-5, 5), "bins": 100},  # (5 - (-5)) / 0.1 = 100 bins
    "Phi": {"range": (-(np.pi), np.pi), "bins": 63},  
    "Invariant": {"range": (50, 3000), "bins": 148}  # (3000 - 50) / 20 = 147.5, rounded to 148 bins
}



<h2> Functions </h2>

<h4>Calculations </h4>

In [3]:
def calculateWeight(num_events, effective_area, lumi):
    return effective_area * lumi / num_events

def calculateSignificance(numSig,numBkg):
    return numSig/np.sqrt(numSig + numBkg)

<h4> Tree Functions </h4>

In [4]:
def openTree(tree_filepath):
    #Open tree and only keep stuff that we need
    
    file = uproot.open(tree_filepath)
    tree = file[tree_name]

    #What we want to keep
    branches_wanted = [
        "MissingET.MET",
        "MissingET.Phi",
        "Jet.PT",
        "Jet.Phi",
        "Jet.Eta",
        "Jet.Mass"
    ]
    df = tree.arrays(branches_wanted,library="pd")

    return df

In [5]:
def applySingleCut(df, mask_function):
    mask = mask_function(df)
    return df[mask]


In [6]:
def applyMultipleCuts(df, mask_functions):
    
    for mask in mask_functions:
        df = applySingleCut(df, mask)

    return df

<h4> Calculate All Weights </h4>

In [7]:
signal_weight_list = []

# Iterate over each signal file
for signal_file in sorted(os.listdir(signal_dir)):  #sorted so that it's the same order each time
    if signal_file.endswith(".root"):
        file_path = os.path.join(signal_dir, signal_file)
        
        # Check if the root file is in the signal_files dictionary
        if signal_file in signal_files:
            signal_df = openTree(file_path)
            signal_temp = signal_df["MissingET.MET"].values

            numSigEvents = len(signal_temp)
            tempSigWeight = calculateWeight(num_events=numSigEvents,effective_area= signal_files[signal_file],lumi=luminescence)
            signal_weight_list.append(tempSigWeight)
        else:
            print(f"Warning: {signal_file} not found in signal_files dictionary.")


background_weight_list = []
for folder_name in sorted(os.listdir(current_dir)):
        folder_path = os.path.join(current_dir, folder_name)

        # Check if the current item is a directory and if its name is in the background_folders dictionary
        if os.path.isdir(folder_path) and folder_name in background_folders:
            cross_section = background_folders[folder_name]

            for root_file in os.listdir(folder_path):
                if root_file.endswith(".root"):
                    file_path = os.path.join(folder_path, root_file)
                    background_df = openTree(file_path) 
                    background_temp = background_df["MissingET.MET"].values #arbitraty branch 

                    numBkgEvents = len(background_temp)
                    tempBkgWeight = calculateWeight(num_events=numBkgEvents, effective_area=cross_section, lumi=luminescence)

                    background_weight_list.append(tempBkgWeight)

In [8]:
print(signal_weight_list)

[0.0108, 0.0228, 0.0015599999999999998, 0.0516]


In [9]:
print(background_weight_list)

[353665.55555555556, 633080.8839779006, 353438.6566648571, 353822.81013783906, 353499.7161825317, 354164.0086047029, 354286.6506715476, 355217.9897329068, 354663.86866736, 354199.04053018126, 353299.1712298357, 2790464.6405610754, 354453.2293986637, 353884.0024706609, 353525.8910060223, 355244.41964285716, 354418.14717672096, 353011.82994454715, 354418.14717672096, 1796044.514106583, 67591.40032191308, 67837.50865406805, 67679.58557175749, 67802.73712806581, 67915.87513477435, 67928.08175212468, 67544.80698529411, 67712.49776048731, 67579.31439664861, 67703.83355512335, 68039.88889746161, 67900.18735723635, 67615.58514580724, 67693.43943503403, 68025.8928801008, 67781.89131715815, 67774.94556167541, 67754.11683355956, 67903.67290367291, 67367.65042906979, 67868.83353428594, 67846.20710878597, 67608.67343027267, 67768.00122953969, 67896.70216861286, 67714.23086767341, 67938.54805988546, 67799.26193588068, 67768.00122953969, 67893.21733774732, 67881.02324626675, 67522.39605931446, 67894.

In [10]:
def getJetsData(dataname, masklist): 
    #used when we are working with J0,J1 stuff (which is often)


    #Signal Processing
    signal_j0_list = []
    signal_j1_list = []

    for signal_file in sorted(os.listdir(signal_dir)):
        if signal_file.endswith(".root"):
            file_path = os.path.join(signal_dir, signal_file)
            
            # Check if the root file is in the signal_files dictionary
            if signal_file in signal_files:
                signal_df = openTree(file_path)

                if(masklist is not None):
                    signal_df = applyMultipleCuts(signal_df,masklist)


                 # Extract and filter jet data
                signal_jets = signal_df["Jet." + dataname].values
                signal_filtered_jets = [entry for entry in signal_jets if len(entry) >= 2]  # At least two entries (j0,j1)

                # Extract J0 and J1
                signal_j0 = np.array([entry[0] for entry in signal_filtered_jets])
                signal_j1 = np.array([entry[1] for entry in signal_filtered_jets])

                signal_j0_list.append(signal_j0)
                signal_j1_list.append(signal_j1)

    

    # Background processing
    background_j0_list = []
    background_j1_list = []

    for folder_name in sorted(os.listdir(current_dir)):
        folder_path = os.path.join(current_dir, folder_name)

        # Check if the current item is a directory and if its name is in the background_folders dictionary
        if os.path.isdir(folder_path) and folder_name in background_folders:

            for root_file in os.listdir(folder_path):
                if root_file.endswith(".root"):
                    file_path = os.path.join(folder_path, root_file)
                    background_df = openTree(file_path)  

                    if(masklist is not None):
                        background_df = applyMultipleCuts(background_df, masklist)

                    # Extract and filter jet data
                    background_jets = background_df["Jet." + dataname].values
                    background_filtered_jets = [entry for entry in background_jets if len(entry) >= 2]  # At least two entries

                    # Extract J0 and J1
                    background_j0 = np.array([entry[0] for entry in background_filtered_jets])
                    background_j1 = np.array([entry[1] for entry in background_filtered_jets])

                  
                    background_j0_list.append(background_j0)
                    background_j1_list.append(background_j1)
                    

    return signal_j0_list, signal_j1_list, background_j0_list, background_j1_list

In [11]:
def PlotJets(binname,dataname,masklist):
    j0_hist_background = Hist(
            axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J0")
        )
    j0_hist_signal = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J0")
    )

    #J1
    j1_hist_background = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J1")
    )
    j1_hist_signal = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J1")
    )

    j0siglist, j1siglist, bkgj0list, bkgj1list,  = getJetsData(dataname,masklist)

    for i in range(len(j0siglist)):
        j0_hist_signal.fill(thedata=j0siglist[i], weight=signal_weight_list[i])
        j1_hist_signal.fill(thedata=j1siglist[i], weight=signal_weight_list[i])


    for i in range(len(background_weight_list)):
        j0_hist_background.fill(thedata=bkgj0list[i], weight=background_weight_list[i])
        j1_hist_background.fill(thedata=bkgj1list[i], weight=background_weight_list[i])
    

    # Create a figure and a set of subplots (2 columns, 1 row)
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot J0
    axs[0].stairs(
        j0_hist_background.values(),
        j0_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )
    axs[0].stairs(
        j0_hist_signal.values(),
        j0_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )
    axs[0].set_xlabel(dataname+'(j0)')
    axs[0].set_ylabel('Counts')
    axs[0].set_yscale('log')
    axs[0].set_title(dataname+'(j0) Distributions')
    axs[0].legend()
    axs[0].grid(True)

    # Plot J1
    axs[1].stairs(
        j1_hist_background.values(),
        j1_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )
    axs[1].stairs(
        j1_hist_signal.values(),
        j1_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )
    axs[1].set_xlabel(dataname+'(j1)')
    axs[1].set_ylabel('Counts')
    axs[1].set_yscale('log')
    axs[1].set_title(dataname+'(j1) Distributions')
    axs[1].legend()
    axs[1].grid(True)

    # Adjust layout
    plt.tight_layout()

    # Show the plot
    plt.show()


    #signal and background events to for the 'cut chart'
    numSigEvents = j0_hist_signal.count().value + j1_hist_signal.count().value
    numBkgEvents = j0_hist_background.count().value + j1_hist_background()

    return numSigEvents, numBkgEvents


<h1> Plot Everything</h1>

<h3> MET <h3>

In [12]:
def PlotMET(masklist):
    met_hist_background = Hist(
        axis.Regular(binning["Transverse"]["bins"], *binning["Transverse"]["range"], name="MET", label="MET")
    )
    met_hist_signal = Hist(
        axis.Regular(binning["Transverse"]["bins"], *binning["Transverse"]["range"], name="MET", label="MET")
    )

    #Signal Data

    counter = 0 #which weight we use 
    for signal_file in sorted(os.listdir(signal_dir)):
        if signal_file.endswith(".root"):
            file_path = os.path.join(signal_dir, signal_file)
            
            # Check if the root file is in the signal_files dictionary
            if signal_file in signal_files:
                signal_df = openTree(file_path)
    
                if(masklist is not None):
                    signal_df = applyMultipleCuts(signal_df,masklist)
    
                signal_met = signal_df["MissingET.MET"].values
                signal_met = ak.flatten(signal_met).to_numpy()
    
                met_hist_signal.fill(MET=signal_met,weight = signal_weight_list[counter])
                counter += 1



    #Background
    counter = 0
    for folder_name in os.listdir(current_dir):
        folder_path = os.path.join(current_dir, folder_name)
            
        # Check if the current item is a directory and if its name is in the background_folders dictionary
        if os.path.isdir(folder_path) and folder_name in background_folders:
            
            
            for root_file in os.listdir(folder_path):
                if root_file.endswith(".root"):
                    file_path = os.path.join(folder_path, root_file)
                    background_df = openTree(file_path)

                    if(masklist is not None):
                        background_df = applyMultipleCuts(background_df, masklist)

                    background_met = background_df["MissingET.MET"].values
                    background_met = ak.flatten(background_met).to_numpy()
                   
                    met_hist_background.fill(MET=background_met,weight =background_weight_list[counter])
                    counter+=1
                
                    




    #Set up histogram
    plt.figure(figsize=(10, 6))

    # Background histogram
    plt.stairs(
        met_hist_background.values(),
        met_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )

    # Signal histogram
    plt.stairs(
        met_hist_signal.values(),
        met_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )

    # Add labels and legend
    plt.xlabel('MET')
    plt.ylabel('Counts')
    plt.yscale('log')
    plt.title('MET Distribution')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()


    #signal and background events to for the 'cut chart'
    numSigEvents = met_hist_signal.count().value 
    numBkgEvents = met_hist_background.count().value

    return numSigEvents, numBkgEvents

In [ ]:
PlotMET(None)

<h3> Phi(Met) </h3>

In [ ]:
def PlotPhiMet(masklist):

    met_phi_hist_background = Hist(
        axis.Regular(binning["Phi"]["bins"], *binning["Phi"]["range"], name="MET_Phi", label="MET Phi (rad)")
    )
    met_phi_hist_signal = Hist(
        axis.Regular(binning["Phi"]["bins"], *binning["Phi"]["range"], name="MET_Phi", label="MET Phi (rad)")
    )


    #Signal
    counter = 0 #which weight we use 
    for signal_file in sorted(os.listdir(signal_dir)):
        if signal_file.endswith(".root"):
            file_path = os.path.join(signal_dir, signal_file)
            
            # Check if the root file is in the signal_files dictionary
            if signal_file in signal_files:
                signal_df = openTree(file_path)
    
                if(masklist is not None):
                    signal_df = applyMultipleCuts(signal_df,masklist)
    
                signal_met_phi = signal_df["MissingET.Phi"].values
                signal_met_phi = ak.flatten(signal_met_phi).to_numpy()
    
                met_phi_hist_signal.fill(MET_Phi=signal_met_phi,weight = signal_weight_list[counter])
                counter += 1

    #Background
    counter = 0
    for folder_name in os.listdir(current_dir):
        folder_path = os.path.join(current_dir, folder_name)
            
        # Check if the current item is a directory and if its name is in the background_folders dictionary
        if os.path.isdir(folder_path) and folder_name in background_folders:
            cross_section = background_folders[folder_name]
            
            for root_file in os.listdir(folder_path):
                if root_file.endswith(".root"):
                    file_path = os.path.join(folder_path, root_file)
                    background_df = openTree(file_path)

                    if(masklist is not None):
                        background_df = applyMultipleCuts(background_df, masklist)

                    background_met_phi = background_df["MissingET.Phi"].values
                    background_met_phi = ak.flatten(background_met_phi).to_numpy()
                    
                    met_phi_hist_background.fill(MET_Phi=background_met_phi,weight =background_weight_list[counter])
                    counter += 1
                    


    #Set up histogram
    plt.figure(figsize=(10, 6))

    # Background histogram
    plt.stairs(
        met_phi_hist_background.values(),
        met_phi_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )

    # Signal histogram
    plt.stairs(
        met_phi_hist_signal.values(),
        met_phi_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )

    # Add labels and legend
    plt.xlabel('MET Phi (rad)')
    plt.ylabel('Counts')
    plt.yscale('log')
    plt.title('MET Phi Distribution')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()


    #signal and background events to for the 'cut chart'
    numSigEvents = met_phi_hist_signal.count().value 
    numBkgEvents = met_phi_hist__background.count().value

    return numSigEvents, numBkgEvents

In [ ]:
PlotPhiMet(None)

<h2> Jets </h2>

<h3> PT(j0) , PT(j1) </h3>

In [ ]:
PlotJets("PT","PT",None)

<h3>Phi(J0,J1)</h3>

In [ ]:
PlotJets("Phi","Phi",None)

<h3> Eta (j0, j1) </h3>

In [ ]:
PlotJets("Eta","Eta",None)

<h3>Eta(j0) * Eta(j1)</h3>

In [ ]:
def PlotEtaEta(masklist):

    etaeta_hist_background = Hist(
        axis.Regular(binning["Eta*Eta"]["bins"], *binning["Eta*Eta"]["range"], name="Eta*Eta", label="Eta*Eta")
    )
    etaeta_hist_signal = Hist(
        axis.Regular(binning["Eta*Eta"]["bins"], *binning["Eta*Eta"]["range"], name="Eta*Eta", label="Eta*Eta")
    )

    j0siglist, j1siglist, bkgj0list, bkgj1list, = getJetsData("Eta",masklist)

    for i in range(len(j0siglist)):
        etaeta_hist_signal.fill(j0siglist[i]*j1siglist[i], weight=signal_weight_list[i])

    for i in range(len(background_weight_list)):
        etaeta_hist_background.fill(bkgj0list[i]*bkgj1list[i], weight = background_weight_list[i])


    #Set up histogram
    plt.figure(figsize=(10, 6))

    # Background histogram
    plt.stairs(
        etaeta_hist_background.values(),
        etaeta_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )

    # Signal histogram
    plt.stairs(
        etaeta_hist_signal.values(),
        etaeta_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )

    # Add labels and legend
    plt.xlabel('Eta(j0) * Eta(j1)')
    plt.ylabel('Counts')
    plt.yscale('log')
    plt.title('Eta * Eta Distribution')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()

    #returns hist itself, for use in the Significance function
    return etaeta_hist_signal, etaeta_hist_background

    

In [ ]:
PlotEtaEta(None)

<h3> Abs(Delta(J0,J1)) </h3>

In [ ]:
def PlotDeltaJets(masklist):

    deltaeta_hist_background = Hist(
        axis.Regular(binning["Delta_Eta"]["bins"], *binning["Delta_Eta"]["range"], name="DeltaEta", label="DeltaEta")
    )

    deltaeta_hist_signal = Hist(
        axis.Regular(binning["Delta_Eta"]["bins"], *binning["Delta_Eta"]["range"], name="DeltaEta", label="DeltaEta")
    )


    j0siglist, j1siglist, bkgj0list, bkgj1list = getJetsData("Eta",masklist)

    
    for i in range(len(j0siglist)):
        deltaeta_hist_signal.fill(np.abs(j0siglist[i]-j1siglist[i]), weight=signal_weight_list[i])

    for i in range(len(background_weight_list)):
        deltaeta_hist_background.fill(np.abs(bkgj0list[i]-bkgj1list[i]), weight = background_weight_list[i])


    #Set up histogram
    plt.figure(figsize=(10, 6))

    # Background histogram
    plt.stairs(
        deltaeta_hist_background.values(),
        deltaeta_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )

    # Signal histogram
    plt.stairs(
        deltaeta_hist_signal.values(),
        deltaeta_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )

    # Add labels and legend
    plt.xlabel('Abs(Delta Eta) (j0, j1)')
    plt.ylabel('Counts')
    plt.yscale('log')
    plt.title('Delta Eta Distribution')
    plt.legend()
    plt.grid(True)


    # Show the plot
    plt.show()

    #returns hist itself, for use in the Significance function
    return deltaeta_hist_signal, deltaeta_hist_background

In [ ]:
PlotDeltaJets(None)

<h3> Invariant Mass (J0 J1) </h3>

In [ ]:
def calc_invariant_mass(pt0, eta0, phi0, m0, pt1, eta1, phi1, m1):
    # Energy and momentum components for Jet0
    pz0 = pt0 * np.sinh(eta0)
    e0 = np.sqrt(pt0**2 + pz0**2 + m0**2)
    px0 = pt0 * np.cos(phi0)
    py0 = pt0 * np.sin(phi0)

    # Energy and momentum components for Jet1
    pz1 = pt1 * np.sinh(eta1)
    e1 = np.sqrt(pt1**2 + pz1**2 + m1**2)
    px1 = pt1 * np.cos(phi1)
    py1 = pt1 * np.sin(phi1)

    # Invariant mass calculation
    e_total = e0 + e1
    px_total = px0 + px1
    py_total = py0 + py1
    pz_total = pz0 + pz1

    m2 = e_total**2 - (px_total**2 + py_total**2 + pz_total**2)
    m2 = np.maximum(m2, 0)  # This will replace negative values with 0
    return np.sqrt(m2)


In [ ]:
def PlotInvariantMass(masklist):

    invariant_hist_background = Hist(
        axis.Regular(binning["Invariant"]["bins"], *binning["Invariant"]["range"], name="Invariant", label="Invariant")
    )

    invariant_hist_signal = Hist(
        axis.Regular(binning["Invariant"]["bins"], *binning["Invariant"]["range"], name="Invariant", label="Invariant")
    )

    #Data needed: pt0, eta0, phi0, m0, pt1, eta1, phi1, m1

    ptj0siglist, ptj1siglist, ptbkgj0list, ptbkgj1list  = getJetsData("PT",masklist)
    etaj0siglist, etaj1siglist, etabkgj0list, etabkgj1list = getJetsData("Eta",masklist)
    phij0siglist, phij1siglist, phibkgj0list, phibkgj1list  = getJetsData("Phi",masklist)
    massj0siglist, massj1siglist, massbkgj0list, massbkgj1list   = getJetsData("Mass",masklist)

    for i in range(len(ptj0siglist)):
        
        invariant_hist_signal.fill(
            calc_invariant_mass(pt0=ptj0siglist[i], eta0=etaj0siglist[i], phi0 = phij0siglist[i],m0=massj0siglist[i],
                        pt1=ptj1siglist[i], eta1=etaj1siglist[i], phi1 = phij1siglist[i],m1=massj1siglist[i]),
            weight = signal_weight_list[i])
    

    for i in range(len(background_weight_list)):
        invariant_hist_background.fill(
        calc_invariant_mass(
                    pt0=ptbkgj0list[i], eta0=etabkgj0list[i], phi0 = phibkgj0list[i],m0=massbkgj0list[i],
                    pt1=ptbkgj1list[i], eta1=etabkgj1list[i], phi1 = phibkgj1list[i],m1=massbkgj1list[i]),
                    weight = background_weight_list[i])



    #Set up histogram
    plt.figure(figsize=(10, 6))

    # Background histogram
    plt.stairs(
        invariant_hist_background.values(),
        invariant_hist_background.axes[0].edges,
        color='blue',
        label='Background',
        linewidth=2
    )

    # Signal histogram
    plt.stairs(
        invariant_hist_signal.values(),
        invariant_hist_signal.axes[0].edges,
        color='red',
        label='Signal',
        linewidth=3
    )

    # Add labels and legend
    plt.xlabel('Invariant Mass j0 j1')
    plt.ylabel('Counts')
    plt.yscale('log')
    plt.title('Invariant Mass Distribution')
    plt.legend()
    plt.grid(True)


    # Show the plot
    plt.show()

    #signal and background events to for the 'cut chart'
    numSigEvents = invariant_hist_signal.sum()
    numBkgEvents = invariant_hist_background.sum()

    return numSigEvents, numBkgEvents

In [ ]:
PlotInvariantMass(None)

<h1> Now we start findsing Significance, making Cuts <h1>

In [ ]:
#This will store the cuts that we make to our data, sequentially
maskList = []

In [ ]:
def GetSignificance(sigEvents, bkgEvents):

    

In [ ]:
def GetSignificances(sighist,bkghist):
    signal_counts = sighist.values()
    bkg_counts = bkghist.values()

    significance = []
    for n_s, n_b in zip(signal_counts, bkg_counts):
        if n_b + n_s > 0:
            S = calculateSignificance(numSig=n_s, numBkg=n_b)
            significance.append(S)
        else:
            significance.append(0)  # If both are zero, significance is zero.

    return np.array(significance)



First we will be using $\eta$($j_0$) *  $\eta$($j_1$) to make this cut. 

In [ ]:
etaeta_sig_hist, etaeta_bkg_hist = PlotEtaEta(None)

In [ ]:
#Let's add the inital state to our cut chart, before making any cuts

currSigEvents = etaeta_sig_hist.sum()
signal_numevents.append(currSigEvents)
currBkgEvents = etaeta_bkg_hist.sum()
background_numevents.append(currBkgEvents)

currSignificance = GetSignificance(currSigEvents,currBkgEvents)
significances.append(currSignificance)
cut_strings.append("Initial State")




<h4> Significance Plot </h4>

In [ ]:
significance = GetSignificances(etaeta_sig_hist,etaeta_bkg_hist)

# Get bin edges from the histogram
bin_edges = etaeta_sig_hist.axes[0].edges

# Plot significance
plt.figure(figsize=(10, 6))
plt.step(bin_edges[:-1], significance, where='mid', label='Significance', color='purple', linewidth=2)

# Add labels and title
plt.xlabel('Eta(j0) * Eta(j1)')
plt.xlim(-17, 17)  #from visual inspection
plt.ylabel('Significance')
plt.yscale('log')
plt.title('Significance Plot')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()



#Top 5 points
# Find the indices of the top 5 significance values
top_indices = np.argsort(significance)[-5:]  # Get indices of the 5 largest values

# Reverse to get them in descending order
top_indices = top_indices[::-1]

# Retrieve the top 5 significance values and their corresponding bin edges
top_significance_values = significance[top_indices]
top_bin_edges = bin_edges[top_indices]

# Print the top 5 significance values and their corresponding bins
print("Top 5 Significance Values and Their Corresponding Bins:")
for i, idx in enumerate(top_indices):
    print(f"Rank {i + 1}: Significance = {top_significance_values[i]:.3f}, Bin = {top_bin_edges[i]:.3f}")


We chose cut to be eta(j0)*eta(j1) >= 0 

In [ ]:
def mask_etaeta_condition(df):
    # Extract Eta for j0 and j1
    eta_j0 = df["Jet.Eta"].apply(lambda x: x[0] if len(x) > 0 else None)
    eta_j1 = df["Jet.Eta"].apply(lambda x: x[1] if len(x) > 1 else None)

    # Create mask where Eta(j0) * Eta(j1) >= 0
    return (eta_j0 * eta_j1) >= 0

maskList.append(mask_etaeta_condition)

Now we will plot invariable mass after making this cut

In [ ]:
currSigEvents, currBkgEvents = PlotInvariantMass(maskList)

In [ ]:
# add cut to cut chart



<b> abs($\Delta$($j_0$,$j_1$)) </b>

Now we will use Delta(J0,J1) to make the next cut. Let's plot it with our current cuts

In [ ]:
deltaeta_sig_hist , deltaeta_bkg_hist = PlotDeltaJets(maskList)

In [ ]:
significance = GetSignificances(deltaeta_sig_hist,deltaeta_bkg_hist)

# Get bin edges from the histogram
bin_edges = deltaeta_sig_hist.axes[0].edges

# Plot significance
plt.figure(figsize=(10, 6))
plt.step(bin_edges[:-1], significance, where='mid', label='Significance', color='purple', linewidth=2)

# Add labels and title
plt.xlabel('Delta(Eta(J0,J1))')
plt.xlim(0, 5)  #from visual inspection
plt.ylabel('Significance')
plt.yscale('log')
plt.title('Significance Plot')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()



#Top 5 points
# Find the indices of the top 5 significance values
top_indices = np.argsort(significance)[-5:]  # Get indices of the 5 largest values

# Reverse to get them in descending order
top_indices = top_indices[::-1]

# Retrieve the top 5 significance values and their corresponding bin edges
top_significance_values = significance[top_indices]
top_bin_edges = bin_edges[top_indices]

# Print the top 5 significance values and their corresponding bins
print("Top 5 Significance Values and Their Corresponding Bins:")
for i, idx in enumerate(top_indices):
    print(f"Rank {i + 1}: Significance = {top_significance_values[i]:.3f}, Bin = {top_bin_edges[i]:.3f}")


Notice that this isn't of use to us, not cut to be made.

<b>Pt($j_0$) </b>

Now we will plot just Pt(j0) as we will use this for our next cut

In [ ]:
def PlotSingleJet(binname, dataname, masklist, j1):  # if j1 is True, plot J1; otherwise, plot J0
    # Create histograms for J0
    j0_hist_background = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J0")
    )
    j0_hist_signal = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J0")
    )

    # Create histograms for J1
    j1_hist_background = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J1")
    )
    j1_hist_signal = Hist(
        axis.Regular(binning[binname]["bins"], *binning[binname]["range"], name="thedata", label=dataname+"J1")
    )

    # Retrieve data
    j0siglist, j1siglist, bkgj0list, bkgj1list = getJetsData(dataname, masklist)

    # Fill histograms
    for i in range(len(j0siglist)):
        j0_hist_signal.fill(thedata=j0siglist[i], weight=signal_weight_list[i])
        j1_hist_signal.fill(thedata=j1siglist[i], weight=signal_weight_list[i])

    for i in range(len(background_weight_list)):
        j0_hist_background.fill(thedata=bkgj0list[i], weight=background_weight_list[i])
        j1_hist_background.fill(thedata=bkgj1list[i], weight=background_weight_list[i])

    # Create a figure and a set of subplots based on the parameter
    if j1:
        # Plot J1 only
        fig, axs = plt.subplots(1, 1, figsize=(8, 6))

        axs.stairs(
            j1_hist_background.values(),
            j1_hist_background.axes[0].edges,
            color='blue',
            label='Background',
            linewidth=2
        )
        axs.stairs(
            j1_hist_signal.values(),
            j1_hist_signal.axes[0].edges,
            color='red',
            label='Signal',
            linewidth=3
        )
        axs.set_xlabel(dataname+'(j1)')
        axs.set_ylabel('Counts')
        axs.set_yscale('log')
        axs.set_title(dataname+'(j1) Distributions')
        axs.legend()
        axs.grid(True)
    else:
        # Plot J0 only
        fig, axs = plt.subplots(1, 1, figsize=(8, 6))

        axs.stairs(
            j0_hist_background.values(),
            j0_hist_background.axes[0].edges,
            color='blue',
            label='Background',
            linewidth=2
        )
        axs.stairs(
            j0_hist_signal.values(),
            j0_hist_signal.axes[0].edges,
            color='red',
            label='Signal',
            linewidth=3
        )
        axs.set_xlabel(dataname+'(j0)')
        axs.set_ylabel('Counts')
        axs.set_yscale('log')
        axs.set_title(dataname+'(j0) Distributions')
        axs.legend()
        axs.grid(True)

    # Adjust layout
    plt.tight_layout()

    # Show the plot
    plt.show()

    if j1:
        return j1_hist_signal, j1_hist_background
    else:
        return j0_hist_signal, j0_hist_background


In [ ]:
ptj0_sig_hist, ptj_bkg_hist = PlotSingleJet("PT","PT",maskList,False)

Now let's plot significannce

In [ ]:
significance = GetSignificances(ptj0_sig_hist, ptj_bkg_hist)

# Get bin edges from the histogram
bin_edges = ptj0_sig_hist.axes[0].edges

# Plot significance
plt.figure(figsize=(10, 6))
plt.step(bin_edges[:-1], significance, where='mid', label='Significance', color='purple', linewidth=2)

# Add labels and title
plt.xlabel('PT(j0)')
#plt.xlim(0, 5)  #from visual inspection
plt.ylabel('Significance')
plt.yscale('log')
plt.title('Significance Plot')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()



#Top 5 points
# Find the indices of the top 5 significance values
top_indices = np.argsort(significance)[-5:]  # Get indices of the 5 largest values

# Reverse to get them in descending order
top_indices = top_indices[::-1]

# Retrieve the top 5 significance values and their corresponding bin edges
top_significance_values = significance[top_indices]
top_bin_edges = bin_edges[top_indices]

# Print the top 5 significance values and their corresponding bins
print("Top 5 Significance Values and Their Corresponding Bins:")
for i, idx in enumerate(top_indices):
    print(f"Rank {i + 1}: Significance = {top_significance_values[i]:.3f}, Bin = {top_bin_edges[i]:.3f}")

We chose our cut to be Pt(j0) > 720

In [ ]:
def mask_pt_j0_condition(df):
    # Extract PT for j0
    pt_j0 = df["Jet.PT"].apply(lambda x: x[0] if len(x) > 0 else None)

    # Create mask where PT(j0) > 510
    return pt_j0 > 510

# Append the mask function to the mask list
maskList.append(mask_pt_j0_condition)


In [ ]:
PlotInvariantMass(maskList)

<b>Pt($j_1$)</b>

<b> $M_{ET}$, <b>

<h1>Step 5: Compile full Cut Chart </h1>

In [ ]:

def print_cut_summary():
    # Print the header
    print(f"{'Cut #':<6} | {'Cut String':<30} | {'Signal Events':<20} | {'Background Events':<20} | {'Significance':<15}")
    print("-" * 100)
    
    # Iterate over the cuts and print each row
    for i in range(num_cuts):
        print(f"{i+1:<6} | {cuts_strings[i]:<30} | {Signal_numevents[i]:<20} | {Background_numevents[i]:<20} | {Significance[i]:<15}")


print_cut_summary()


<h2> CODE TESTING </h2>

In [ ]:
def mask_pt_j0_greater_than_30(df):
    # Extract PT for j0
    pt_j0 = df["Jet.PT"].apply(lambda x: x[0] if len(x) > 0 else None)
    
    # Create mask where PT(j0) > 30
    return pt_j0 > 550


def mask_eta_condition(df):
    # Extract Eta for j0 and j1
    eta_j0 = df["Jet.Eta"].apply(lambda x: x[0] if len(x) > 0 else None)
    eta_j1 = df["Jet.Eta"].apply(lambda x: x[1] if len(x) > 1 else None)

    # Create mask where Eta(j0) * Eta(j1) >= 0
    return (eta_j0 * eta_j1) >= 0